# Q2

En esta parte, la tarea consiste en llamar a MongoDB para realizar geoqueries y representar los datos geoespaciales con folium, mostrando en un mapa la geolocalizacion de la nueva oficina.

In [1]:
# Tu codigo aqui
from pymongo import MongoClient,GEOSPHERE
import folium
from bson.son import SON
import pandas as pd

In [13]:
client = MongoClient('mongodb://localhost:27017/')
db = client.db_geoloc
colec = db.geop

In [15]:
colec.insert_many([loads(one) for one in open('geop.json','r')][0]);

In [ ]:
df = pd.read_json('geop.json')
df

In [ ]:
db.geop.create_index([('Geolocation', pymongo.GEOSPHERE)])

In [8]:
def findNear(location, ratio=2000):
    return colec.find(
                        {'Geolocation':
                             {'$near':{'$geometry':location, '$maxDistance':ratio}}
                        }
                        )
df['nearest'] = df['Geolocation'].apply(lambda x: list(findNear(x)))

In [11]:
puntos = folium.Map([41.109534,-73.72399],tiles='Stamen Toner', zoom_start=10) 
puntos

In [ ]:
for p in df.index:
    folium.Marker([df.loc[p,'latitude'], df.loc[p,'longitude']],icon=folium.Icon(icon='cloud'),
                popup=df.loc[p,'name']).add_to(points)

In [ ]:
puntos